## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Oct. 7th Oct. 14th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,October-7-2024,Condo/Co-op,1425 Brickell Ave Unit 54A,Miami,FL,33131.0,2150000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1425-Brickell-...,MARMLS,A11649240,N,Y,25.759132,-80.191829


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
47,PAST SALE,2024-10-12,Condo/Co-op,92 SW 3rd St #4804,Miami,FL,33130.0,706000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/92-SW-3rd-St-3...,MARMLS,A11614190,N,Y,25.769593,-80.195047
25,PAST SALE,2024-10-12,Condo/Co-op,854 NW 87th Ave #101,Miami,FL,33172.0,315000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/854-NW-87th-Av...,MARMLS,A11640505,N,Y,25.779318,-80.337169
30,PAST SALE,2024-10-11,Condo/Co-op,1001 7th St #203,Miami Beach,FL,33139.0,220000.0,0.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1001-7th...,MARMLS,A11629666,N,Y,25.777125,-80.138717
17,PAST SALE,2024-10-11,Condo/Co-op,3350 SW 27th Ave #1601,Miami,FL,33133.0,2100000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3350-SW-27th-A...,MARMLS,A11640355,N,Y,25.730469,-80.237786
61,PAST SALE,2024-10-11,Condo/Co-op,19400 Turnberry Way #1911,Aventura,FL,33180.0,895000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/19400-Turnberr...,MARMLS,A11582219,N,Y,25.955279,-80.128017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,PAST SALE,2024-10-07,Condo/Co-op,426 NE 77th St Rd #2,Miami,FL,33138.0,269000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/426-NE-77th-St...,MARMLS,A11606196,N,Y,25.846303,-80.187771
48,PAST SALE,2024-10-07,Condo/Co-op,3600 Yacht Club Dr #704,Aventura,FL,33180.0,480000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3600-Yacht-Clu...,MARMLS,A11595055,N,Y,25.973077,-80.129543
52,PAST SALE,2024-10-07,Condo/Co-op,488 NE 18th St #2100,Miami,FL,33132.0,745000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/488-NE-18th-St...,MARMLS,A11587429,N,Y,25.793508,-80.186688
55,PAST SALE,2024-10-07,Condo/Co-op,6000 Island Blvd #2206,Aventura,FL,33160.0,3159000.0,4.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11567546,N,Y,25.942761,-80.135433


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [11]:
df_top_ten = df.head(10)

## Collect Agent Information

In [12]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [13]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [14]:
response_df = pd.DataFrame(response_list)

In [15]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [16]:
df_top_ten = merged_df

## Current Week's Values

In [17]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

83


In [18]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$937,542


In [19]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$561


In [20]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$77,816,000


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [21]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
120
Input Previous Week Condo Average Sales Price:
625291
Input Previous Week Condo Average PSF:
501
Input Previous Week Condo Sales total (ex: 198_000_000)
75_000_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [22]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [23]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [24]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [25]:
pd.set_option('display.max_columns',None)

In [26]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-10,Condo/Co-op,2669 S Bayshore Dr Unit 1703N,Miami,FL,33133.0,8525000.0,6.0,6.0,Grove at Grand Bay,5380.0,NaN,2016.0,NaN,1585.0,8362.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,MARMLS,A11573491,N,Y,25.730676,-80.237117,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,Marcello Agostini,"AG Real Estate Advisors, LLC",Fernanda Souza,"AG Real Estate Advisors, LLC",David Cabarrocas,Coldwell Banker Realty,Judith Zeder,Coldwell Banker Realty,1,orange


In [27]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [28]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-10,Condo/Co-op,2669 S Bayshore Dr Unit 1703N,Miami,FL,33133.0,8525000.0,6.0,6.0,Grove at Grand Bay,5380.0,NaN,2016.0,NaN,1585.0,8362.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,MARMLS,A11573491,N,Y,25.730676,-80.237117,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,Marcello Agostini,"AG Real Estate Advisors, LLC",Fernanda Souza,"AG Real Estate Advisors, LLC",David Cabarrocas,Coldwell Banker Realty,Judith Zeder,Coldwell Banker Realty,1,orange
1,PAST SALE,2024-10-07,Condo/Co-op,17749 Collins Ave #3001,Sunny Isles Beach,FL,33160.0,6900000.0,3.0,6.5,Mansions at Acqualina,4609.0,NaN,2015.0,NaN,1497.0,10300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11324624,N,Y,25.940065,-80.120264,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Lana Bell,Douglas Elliman,None,None,Sylvia Dilsen,Charles Rutenberg Realty Fort,None,None,2,blue
2,PAST SALE,2024-10-10,Condo/Co-op,3300 NE 188th St #318,Aventura,FL,33180.0,3275000.0,4.0,6.5,ECHO CONDO,3801.0,NaN,2015.0,NaN,862.0,4227.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3300-NE-188th-...,MARMLS,A11538465,N,Y,25.948958,-80.136544,https://www.redfin.com/FL/Miami/3300-NE-188th-...,Juliana Tobon,Cervera Real Estate Inc,Alexandra Goeseke,Cervera Real Estate Inc.,Leila Chikhladze,Lifestyle International Realty,None,None,3,blue
3,PAST SALE,2024-10-07,Condo/Co-op,6000 Island Blvd #2206,Aventura,FL,33160.0,3159000.0,4.0,3.5,BELLAMARE AT WILLIAMS ISL,3528.0,NaN,2005.0,NaN,895.0,3208.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11567546,N,Y,25.942761,-80.135433,https://www.redfin.com/FL/Aventura/6000-Island...,Rafael Szydlowski,Coldwell Banker Realty,Natalia Abecasis,Coldwell Banker Realty,Jackie Cabral,Dezer Platinum Realty LLC,None,None,4,blue
4,PAST SALE,2024-10-10,Condo/Co-op,1000 Brickell Plz Ph 6008,Miami,FL,33131.0,2970000.0,3.0,3.5,BRICKELL FLATIRON CONDO,1912.0,NaN,2019.0,NaN,1553.0,2151.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11473995,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Fabio Lopes,Coldwell Banker Realty,Felise Eber,Coldwell Banker Realty,Elizabeth Fernandez,Privee Real Estate INC,None,None,5,blue
5,PAST SALE,2024-10-10,Condo/Co-op,700 NE 26th Ter #5602,Miami,FL,33137.0,2800000.0,2.0,3.5,Missoni Baia,2464.0,NaN,2023.0,NaN,1136.0,3370.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11476703,N,Y,25.802198,-80.185790,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,Maggie Munoz,Fortune International Realty,None,None,Michael Light,Douglas Elliman,None,None,6,blue
6,PAST SALE,2024-10-10,Condo/Co-op,6899 Collins Ave #1006,Miami Beach,FL,33141.0,2700000.0,3.0,3.0,NORTH CARILLON BEACH COND,2050.0,NaN,2008.0,NaN,1317.0,5689.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6899-Col...,MARMLS,A11529878,N,Y,25.853732,-80.120089,https://www.redfin.com/FL/Miami-Beach/6899-Col...,Francisco Neri,The Corcoran Group,None,None,Shelly Bressler,Douglas Elliman,Kayce Driscoll,Douglas Elliman,7,blue
7,PAST SALE,2024-10-08,Condo/Co-op,9 Island Ave #508,Miami Beach,FL,33139.0,2203750.0,2.0,2.5,NINE ISLAND AVENUE CONDO,1837.0,NaN,1981.0,NaN,1200.0,2556.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/9-Island...,MARMLS,A11625325,N,Y,25.790337,-80.148773,https://www.redfin.com/FL/Miami-Beach/9-Island...,William Hahne,Coldwell Banker Realty,Jeremy Vasquez,Coldwell Banker Realty,Jessica Adams,Douglas Elliman,None,None,8,blue
8,PAST SALE,2024-10-07,Condo/Co-op,1425 Brickell

In [29]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [30]:
m.save('index.html')

## Data snagger

In [31]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-10-10,Condo/Co-op,2669 S Bayshore Dr Unit 1703N,Miami,FL,33133.0,8525000.0,6.0,6.0,Grove at Grand Bay,5380.0,NaN,2016.0,NaN,1585.0,8362.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,MARMLS,A11573491,N,Y,25.730676,-80.237117,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,Marcello Agostini,"AG Real Estate Advisors, LLC",Fernanda Souza,"AG Real Estate Advisors, LLC",David Cabarrocas,Coldwell Banker Realty,Judith Zeder,Coldwell Banker Realty,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [33]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MD_County_condo_sales_week_ending_10142024


# CREATE TEMPLATE 

In [34]:
muni_set = set(df_top_ten['CITY'])

In [35]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [36]:
df_top_ten.reset_index(inplace=True,drop=True)

In [37]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [38]:
top_sale

'Grove at Grand Bay at 2669 S Bayshore Dr Unit 1703N in Miami'

In [39]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [40]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [41]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-10,Condo/Co-op,2669 S Bayshore Dr Unit 1703N,Miami,FL,33133.0,8525000.0,6.0,6.0,Grove at Grand Bay,5380.0,NaN,2016.0,NaN,1585.0,8362.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,MARMLS,A11573491,N,Y,25.730676,-80.237117,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,Marcello Agostini,"AG Real Estate Advisors, LLC",Fernanda Souza,"AG Real Estate Advisors, LLC",David Cabarrocas,Coldwell Banker Realty,Judith Zeder,Coldwell Banker Realty,1,orange,Grove at Grand Bay at 2669 S Bayshore Dr Unit ...
1,PAST SALE,2024-10-07,Condo/Co-op,17749 Collins Ave #3001,Sunny Isles Beach,FL,33160.0,6900000.0,3.0,6.5,Mansions at Acqualina,4609.0,NaN,2015.0,NaN,1497.0,10300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11324624,N,Y,25.940065,-80.120264,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Lana Bell,Douglas Elliman,None,None,Sylvia Dilsen,Charles Rutenberg Realty Fort,None,None,2,blue,Mansions at Acqualina at 17749 Collins Ave #30...
2,PAST SALE,2024-10-10,Condo/Co-op,3300 NE 188th St #318,Aventura,FL,33180.0,3275000.0,4.0,6.5,ECHO CONDO,3801.0,NaN,2015.0,NaN,862.0,4227.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3300-NE-188th-...,MARMLS,A11538465,N,Y,25.948958,-80.136544,https://www.redfin.com/FL/Miami/3300-NE-188th-...,Juliana Tobon,Cervera Real Estate Inc,Alexandra Goeseke,Cervera Real Estate Inc.,Leila Chikhladze,Lifestyle International Realty,None,None,3,blue,ECHO CONDO at 3300 NE 188th St #318 in Aventura
3,PAST SALE,2024-10-07,Condo/Co-op,6000 Island Blvd #2206,Aventura,FL,33160.0,3159000.0,4.0,3.5,BELLAMARE AT WILLIAMS ISL,3528.0,NaN,2005.0,NaN,895.0,3208.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11567546,N,Y,25.942761,-80.135433,https://www.redfin.com/FL/Aventura/6000-Island...,Rafael Szydlowski,Coldwell Banker Realty,Natalia Abecasis,Coldwell Banker Realty,Jackie Cabral,Dezer Platinum Realty LLC,None,None,4,blue,BELLAMARE AT WILLIAMS ISL at 6000 Island Blvd ...
4,PAST SALE,2024-10-10,Condo/Co-op,1000 Brickell Plz Ph 6008,Miami,FL,33131.0,2970000.0,3.0,3.5,BRICKELL FLATIRON CONDO,1912.0,NaN,2019.0,NaN,1553.0,2151.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11473995,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Fabio Lopes,Coldwell Banker Realty,Felise Eber,Coldwell Banker Realty,Elizabeth Fernandez,Privee Real Estate INC,None,None,5,blue,BRICKELL FLATIRON CONDO at 1000 Brickell Plz P...
5,PAST SALE,2024-10-10,Condo/Co-op,700 NE 26th Ter #5602,Miami,FL,33137.0,2800000.0,2.0,3.5,Missoni Baia,2464.0,NaN,2023.0,NaN,1136.0,3370.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11476703,N,Y,25.802198,-80.185790,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,Maggie Munoz,Fortune International Realty,None,None,Michael Light,Douglas Elliman,None,None,6,blue,Missoni Baia at 700 NE 26th Ter #5602 in Miami
6,PAST SALE,2024-10-10,Condo/Co-op,6899 Collins Ave #1006,Miami Beach,FL,33141.0,2700000.0,3.0,3.0,NORTH CARILLON BEACH COND,2050.0,NaN,2008.0,NaN,1317.0,5689.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6899-Col...,MARMLS,A11529878,N,Y,25.853732,-80.120089,https://www.redfin.com/FL/Miami-Beach/6899-Col...,Francisco Neri,The Corcoran Group,None,None,Shelly Bressler,Douglas Elliman,Kayce Driscoll,Douglas Elliman,7,blue,NORTH CARILLON BEACH COND at 6899 Collins Ave ...
7,PAST SALE,2024-10-08,Condo/Co-op,9 Island Ave #508,Miami Beach,FL,33139.0,2203750.0,2.0,2.5,NINE ISLAND AVENUE CONDO,1837.0

In [42]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Grove at Grand Bay closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,100,000M to $8,525,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Miami-Dade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Miami, Sunny Isles Beach, Aventura

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 83 condo sales totaling $77,816,000 million from Oct. 7th to Oct. 14th. The previous week, brokers closed 120 condo sales totaling $75,000,000.

Last week’s units sold for an average of $937,542, higher than the $625,291 average price from the previous week. The average price per square 

In [43]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [44]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [45]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-10,Condo/Co-op,2669 S Bayshore Dr Unit 1703N,Miami,FL,33133.0,8525000.0,6.0,6.0,Grove at Grand Bay,5380.0,NaN,2016.0,NaN,1585.0,8362.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,MARMLS,A11573491,N,Y,25.730676,-80.237117,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,Marcello Agostini,"AG Real Estate Advisors, LLC",Fernanda Souza,"AG Real Estate Advisors, LLC",David Cabarrocas,Coldwell Banker Realty,Judith Zeder,Coldwell Banker Realty,1,orange,Grove at Grand Bay at 2669 S Bayshore Dr Unit ...
1,PAST SALE,2024-10-07,Condo/Co-op,17749 Collins Ave #3001,Sunny Isles Beach,FL,33160.0,6900000.0,3.0,6.5,Mansions at Acqualina,4609.0,NaN,2015.0,NaN,1497.0,10300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11324624,N,Y,25.940065,-80.120264,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Lana Bell,Douglas Elliman,None,None,Sylvia Dilsen,Charles Rutenberg Realty Fort,None,None,2,blue,Mansions at Acqualina at 17749 Collins Ave #30...
2,PAST SALE,2024-10-10,Condo/Co-op,3300 NE 188th St #318,Aventura,FL,33180.0,3275000.0,4.0,6.5,ECHO CONDO,3801.0,NaN,2015.0,NaN,862.0,4227.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3300-NE-188th-...,MARMLS,A11538465,N,Y,25.948958,-80.136544,https://www.redfin.com/FL/Miami/3300-NE-188th-...,Juliana Tobon,Cervera Real Estate Inc,Alexandra Goeseke,Cervera Real Estate Inc.,Leila Chikhladze,Lifestyle International Realty,None,None,3,blue,ECHO CONDO at 3300 NE 188th St #318 in Aventura
3,PAST SALE,2024-10-07,Condo/Co-op,6000 Island Blvd #2206,Aventura,FL,33160.0,3159000.0,4.0,3.5,BELLAMARE AT WILLIAMS ISL,3528.0,NaN,2005.0,NaN,895.0,3208.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11567546,N,Y,25.942761,-80.135433,https://www.redfin.com/FL/Aventura/6000-Island...,Rafael Szydlowski,Coldwell Banker Realty,Natalia Abecasis,Coldwell Banker Realty,Jackie Cabral,Dezer Platinum Realty LLC,None,None,4,blue,BELLAMARE AT WILLIAMS ISL at 6000 Island Blvd ...
4,PAST SALE,2024-10-10,Condo/Co-op,1000 Brickell Plz Ph 6008,Miami,FL,33131.0,2970000.0,3.0,3.5,BRICKELL FLATIRON CONDO,1912.0,NaN,2019.0,NaN,1553.0,2151.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11473995,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Fabio Lopes,Coldwell Banker Realty,Felise Eber,Coldwell Banker Realty,Elizabeth Fernandez,Privee Real Estate INC,None,None,5,blue,BRICKELL FLATIRON CONDO at 1000 Brickell Plz P...
5,PAST SALE,2024-10-10,Condo/Co-op,700 NE 26th Ter #5602,Miami,FL,33137.0,2800000.0,2.0,3.5,Missoni Baia,2464.0,NaN,2023.0,NaN,1136.0,3370.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11476703,N,Y,25.802198,-80.185790,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,Maggie Munoz,Fortune International Realty,None,None,Michael Light,Douglas Elliman,None,None,6,blue,Missoni Baia at 700 NE 26th Ter #5602 in Miami
6,PAST SALE,2024-10-10,Condo/Co-op,6899 Collins Ave #1006,Miami Beach,FL,33141.0,2700000.0,3.0,3.0,NORTH CARILLON BEACH COND,2050.0,NaN,2008.0,NaN,1317.0,5689.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6899-Col...,MARMLS,A11529878,N,Y,25.853732,-80.120089,https://www.redfin.com/FL/Miami-Beach/6899-Col...,Francisco Neri,The Corcoran Group,None,None,Shelly Bressler,Douglas Elliman,Kayce Driscoll,Douglas Elliman,7,blue,NORTH CARILLON BEACH COND at 6899 Collins Ave ...
7,PAST SALE,2024-10-08,Condo/Co-op,9 Island Ave #508,Miami Beach,FL,33139.0,2203750.0,2.0,2.5,NINE ISLAND AVENUE CONDO,1837.0

In [47]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: Grove at Grand Bay closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,100,000M to $8,525,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Miami-Dade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Miami Beach, Miami, Sunny Isles Beach, Aventura

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 83 condo sales totaling $77,816,000 million from Oct. 7th to Oct. 14th. The previous week, brokers closed 120 condo sales totaling $75,000,000.

Last week’s units sold for an average of $937,542, higher than the $625,291 average price from the previous week. The average price per square 

In [48]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Miami/2669-S-Bayshore-Dr-33133/unit-1703N/home/112900370


In [60]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Sunny-Isles-Beach/17749-Collins-Ave-33160/unit-3001/home/101345438


In [61]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami/3350-SW-27th-Ave-33133/unit-1601/home/43286969


In [62]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami/1425-Brickell-Ave-33131/unit-54A/home/43295736


In [63]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami/2669-S-Bayshore-Dr-33133/unit-1703N/home/112900370


In [64]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Miami/3300-NE-188th-St-33180/unit-318/home/56743973


In [65]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-10-10,Condo/Co-op,2669 S Bayshore Dr Unit 1703N,Miami,FL,33133.0,8525000.0,6.0,6.0,Grove at Grand Bay,5380.0,NaN,2016.0,NaN,1585.0,8362.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,MARMLS,A11573491,N,Y,25.730676,-80.237117,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,Marcello Agostini,"AG Real Estate Advisors, LLC",Fernanda Souza,"AG Real Estate Advisors, LLC",David Cabarrocas,Coldwell Banker Realty,Judith Zeder,Coldwell Banker Realty,1,orange,Grove at Grand Bay at 2669 S Bayshore Dr Unit ...
1,PAST SALE,2024-10-07,Condo/Co-op,17749 Collins Ave #3001,Sunny Isles Beach,FL,33160.0,6900000.0,3.0,6.5,Mansions at Acqualina,4609.0,NaN,2015.0,NaN,1497.0,10300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11324624,N,Y,25.940065,-80.120264,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Lana Bell,Douglas Elliman,None,None,Sylvia Dilsen,Charles Rutenberg Realty Fort,None,None,2,blue,Mansions at Acqualina at 17749 Collins Ave #30...
2,PAST SALE,2024-10-10,Condo/Co-op,3300 NE 188th St #318,Aventura,FL,33180.0,3275000.0,4.0,6.5,ECHO CONDO,3801.0,NaN,2015.0,NaN,862.0,4227.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3300-NE-188th-...,MARMLS,A11538465,N,Y,25.948958,-80.136544,https://www.redfin.com/FL/Miami/3300-NE-188th-...,Juliana Tobon,Cervera Real Estate Inc,Alexandra Goeseke,Cervera Real Estate Inc.,Leila Chikhladze,Lifestyle International Realty,None,None,3,blue,ECHO CONDO at 3300 NE 188th St #318 in Aventura
3,PAST SALE,2024-10-07,Condo/Co-op,6000 Island Blvd #2206,Aventura,FL,33160.0,3159000.0,4.0,3.5,BELLAMARE AT WILLIAMS ISL,3528.0,NaN,2005.0,NaN,895.0,3208.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/6000-Island...,MARMLS,A11567546,N,Y,25.942761,-80.135433,https://www.redfin.com/FL/Aventura/6000-Island...,Rafael Szydlowski,Coldwell Banker Realty,Natalia Abecasis,Coldwell Banker Realty,Jackie Cabral,Dezer Platinum Realty LLC,None,None,4,blue,BELLAMARE AT WILLIAMS ISL at 6000 Island Blvd ...
4,PAST SALE,2024-10-10,Condo/Co-op,1000 Brickell Plz Ph 6008,Miami,FL,33131.0,2970000.0,3.0,3.5,BRICKELL FLATIRON CONDO,1912.0,NaN,2019.0,NaN,1553.0,2151.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11473995,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Fabio Lopes,Coldwell Banker Realty,Felise Eber,Coldwell Banker Realty,Elizabeth Fernandez,Privee Real Estate INC,None,None,5,blue,BRICKELL FLATIRON CONDO at 1000 Brickell Plz P...
5,PAST SALE,2024-10-10,Condo/Co-op,700 NE 26th Ter #5602,Miami,FL,33137.0,2800000.0,2.0,3.5,Missoni Baia,2464.0,NaN,2023.0,NaN,1136.0,3370.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11476703,N,Y,25.802198,-80.185790,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,Maggie Munoz,Fortune International Realty,None,None,Michael Light,Douglas Elliman,None,None,6,blue,Missoni Baia at 700 NE 26th Ter #5602 in Miami
6,PAST SALE,2024-10-10,Condo/Co-op,6899 Collins Ave #1006,Miami Beach,FL,33141.0,2700000.0,3.0,3.0,NORTH CARILLON BEACH COND,2050.0,NaN,2008.0,NaN,1317.0,5689.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6899-Col...,MARMLS,A11529878,N,Y,25.853732,-80.120089,https://www.redfin.com/FL/Miami-Beach/6899-Col...,Francisco Neri,The Corcoran Group,None,None,Shelly Bressler,Douglas Elliman,Kayce Driscoll,Douglas Elliman,7,blue,NORTH CARILLON BEACH COND at 6899 Collins Ave ...
7,PAST SALE,2024-10-08,Condo/Co-op,9 Island Ave #508,Miami Beach,FL,33139.0,2203750.0,2.0,2.5,NINE ISLAND AVENUE CONDO,1837.0

## Time on Market Calculator

In [66]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 1, 9) ## List (Earlier) date
date2 = datetime(2024, 10, 7) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

272


## Clean CSV for Datawrapper Chart

In [67]:
chart_df = df_top_ten

In [68]:
chart_df = chart_df.fillna(" ")

In [69]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [70]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [71]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [72]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [73]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [74]:
csv_date_string = today.strftime("%m_%d_%Y")

In [75]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")